# Project Introduction

__[Department of Transportation's Responsibilities](https://www.google.com/search?client=safari&rls=en&q=department+of+transportation+responsibilities&ie=UTF-8&oe=UTF-8)__

Through this dataset, I have identified the three following patterns:
1. IDEA 1 w/ LOCAL LINK
2. IDEA 2 w/ LOCAL LINK
3. IDEA 3 w/ LOCAL LINK

## Resources:

- __[Kaggle-Full](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents/data)__

- __[Kaggle-Sampled](https://drive.google.com/file/d/1U3u8QYzLjnEaSurtZfSAS_oh9AT2Mn8X/edit)__

- __[Bing API](https://learn.microsoft.com/en-us/bingmaps/rest-services/traffic/get-traffic-incidents#supported-http-methods)__

- __[MQuest API](https://developer.mapquest.com/documentation/api/traffic/incidents/get.html)__

- __['A Countrywide Traffic Accident Dataset'](https://arxiv.org/pdf/1906.05409)__

- __['Accident Risk Prediction based on Heterogenous Sparse Data: New Dataset & Insights](https://arxiv.org/pdf/1909.09638)__

- __['Census Data'](https://www.census.gov/acs/www/data/data-tables-and-tools/subject-tables/)__

# 1-Setup Environment

## Libraries

In [ ]:
#Utilities
import warnings
import os

# Data Basics
import pandas as pd
import numpy as np
import missingno as msgno

#Time Data
from datetime import date, timedelta


#PySpark
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics
import pyspark.sql.functions as F

#Visuals
import matplotlib.pyplot as plt
import seaborn as sns


# Statistical Analysis
from scipy.stats import zscore, f_oneway, chi2_contingency, chisquare
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Spatial Tools
import geopy
import geopy.distance
import census
import requests

## Info on New Libraries
For improved analysis, these libraries were included, but not covered in the course material:

- __[Geopy:  ](https://geopy.readthedocs.io/en/stable/)__

- Census:
    * __[PyPi](https://pypi.org/project/census/)__
    * __[API](https://www.census.gov/data/developers/guidance/api-user-guide.html)__

- MissingNo:  
    * __[Library](https://github.com/ResidentMario/missingno)__
    
    * __[Tutorial](https://www.geeksforgeeks.org/python-visualize-missing-values-nan-values-using-missingno-library/)__

- __[Yellowbrick](https://www.scikit-yb.org/en/latest/)__

## Settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)
_LITE_SWITCH_ = False
_SPARK_ = False

## Custom Functions

In [ ]:
# For High-level data exploration
def count_outliers(df_col,cap=3):
    zs = zscore(df_col)
    return df_col[zs > cap].shape[0]

## Load Dataset(s)

### Main Dataset
Will depend on settings: lite_switch gives option to use the scaled down dataset.  _SPARK_ will give option of loading with pyspark (which will also determine functionality through out project).

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('US_Accidents_March23.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('US_Accidents_March23_sampled_500k.csv')
    else:
        data = pd.read_csv('US_Accidents_March23.csv')    

### Other User-Built Datasets

# 2-Initial EDA

## Schema & Feature Basics

In [ ]:
if _SPARK_:
    data.printSchema()
    print("Features: ",len(data.columns))
    print("Entries:  ",data.count())
else:
    data.info()

Notes on dimensionality and data types as well as discussion of each feature based on research.

In [1]:
# Organize features into global variables:

## Missing Values

In [ ]:
if _SPARK_:
    missing = data.select(*[F.sum(F.isnull(F.col(c)).cast("int")).alias(c) for c in data.columns])
    print(missing.show(vertical=True))
else:
    print(data.isna().sum().sort_values(ascending=False))

Note on missing data generalities before deeper explorations

## Duplicates

There are no duplicates to deal with:

In [ ]:
if _SPARK_:
    duplicates = int(data.count() - data.dropDuplicates().count())
    duplicates.show()
else:
    print(data.duplicated().sum())

## Outliers

In [ ]:
if _SPARK_:
    pass
else:
    print(pd.DataFrame({c:{z:count_outliers(data[c],z) for z in [3,5,10,15,20]} for c in _NUMERICS_}).T)

Distance seems to be the only feature in the original dataset (lite or full) that has significant outliers.

## High-Level Features

### Nature of Accident

We will eventually create another feature to describe the nature of any accident but for now, we have severity (a discrete variable) and distance.

Any comment on severity

Any comment on distance.

### Time Features

To explore properly, we need to convert to datetime type (from pandas).  Also, need to convert into seconds and then into hours.

It is required to set the 'format' parameter to "mixed".  (Too many observations to understand why / explore observations setting off the error.)


In [ ]:
start = pd.to_datetime(data['Start_Time'],format='mixed')
end = pd.to_datetime(data['End_Time'],format='mixed')
time_change = (end - start).astype('timedelta64[s]').dt.seconds/360
time_change

In [ ]:
time_change.describe()

Comments & Actions

### Address Features

In [ ]:
data['Country'].value_counts()

Comments & Actions

#### Address: State, City, Zip, Aiport Code

In [2]:
# Check for bad strings
unclean = data['State'].str.startswith(' ').sum() + data['State'].str.endswith(' ').sum()
unclean

NameError: name 'data' is not defined

In [ ]:
data['State'].value_counts(dropna=False,normalize=True)

Comments & Actions.  Concerns about balance?

In [3]:
# Same Double Check
unclean = data['City'].str.startswith(' ').sum() + data['City'].str.endswith(' ').sum()
unclean

NameError: name 'data' is not defined

In [ ]:
data['City'].value_counts(dropna=False,normalize=True).sort_values(ascending=False).head(25)

Comments & Actions.  Concerns about balance?

In [ ]:
unclean = data['Zipcode'].str.startswith(' ').sum() + data['Zipcode'].str.endswith(' ').sum()
unclean

In [ ]:
data['Zipcode'].value_counts(dropna=False,normalize=True).sort_values(ascending=False)

Comments & Actions.  Concerns about balance?

In [ ]:
unclean = data['Airport_Code'].str.startswith(' ').sum() + data['Airport_Code'].str.endswith(' ').sum()
unclean

In [ ]:
data['Airport_Code'].value_counts(dropna=False,normalize=True).sort_values(ascending=False)

#### Street Type

In [ ]:
unclean = data['Street'].str.startswith(' ').sum() + data['Street'].str.endswith(' ').sum()
unclean

In [ ]:
streets = data['Street'].str.strip()

In [ ]:
unclean = streets.str.startswith(' ').sum() + streets.str.endswith(' ').sum()
unclean

In [ ]:
streets.value_counts(dropna=False,normalize=True).sort_values(ascending=False).head(25)

Comments and Action Recommendations but experiment first based on the namking conventions discussed in these resources:

Resources on Road Types & Naming Syntax:
__[Google](https://www.google.com/search?q=us+highways+and+interstates&client=safari&sca_esv=a2d3658fd380a756&rls=en&ei=t65JaN2uMMKh5NoPo6mVkQo&oq=US+highways+vs%C2%A0&gs_lp=Egxnd3Mtd2l6LXNlcnAaAhgCIhBVUyBoaWdod2F5cyB2c8KgKgIIADILEAAYgAQYkQIYigUyBxAAGIAEGAoyBRAAGIAEMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgsQABiABBiGAxiKBTILEAAYgAQYhgMYigUyCxAAGIAEGIYDGIoFMggQABiABBiiBEiDTFDVDFjWP3AFeAGQAQCYAXOgAdcKqgEEMTYuMbgBAcgBAPgBAZgCFqACgwzCAgoQABiwAxjWBBhHwgIKEAAYgAQYQxiKBcICERAuGIAEGLEDGNEDGIMBGMcBwgILEAAYgAQYsQMYgwHCAgUQLhiABMICDhAuGIAEGLEDGIMBGNQCwgILEAAYgAQYsQMYiwPCAggQABiABBiLA8ICCBAAGIAEGLEDwgIKEC4YgAQYQxiKBcICERAAGIAEGJECGLEDGIMBGIoFwgIEEAAYA8ICBRAhGKABmAMAiAYBkAYIkgcEMjEuMaAHl26yBwQxNi4xuAfmC8IHBjItMTYuNsgHhwE&sclient=gws-wiz-serp)__ on the difference between US numbered highways and US interstate highway system

https://99percentinvisible.org/article/beyond-streets-avenues-simple-visual-guide-different-types-roads/


Federal Roads.  Highways & Interstates

In [ ]:
highways = streets.str.startswith('US').sum()+ streets.str.startswith('United States').sum()

interstates = streets.str.startswith('I-').sum() + streets.str.startswith('Interstate').sum()

fed_roads = highways + interstates

msg = f'There are roughly {fed_roads} incidents which occured on a federal roads:  '
msg += f'{highways} on a numbered highways and {interstates} interstates.'

print(msg)

Routes

### Weather

#### Quantitative Features

In [ ]:
weather_factors = ['Temperature(F)','Wind_Chill(F)',
                   'Humidity(%)','Pressure(in)',
                   'Visibility(mi)','Wind_Speed(mph)',
                   'Precipitation(in)']

data[weather_factors].describe()

Comments & actions (if needed.)

#### Categorical Features

In [ ]:
weather_descriptions = data['Weather_Condition'].unique()
print(f'There are {weather_descriptions.shape[0]} ways that the weather is described in this dataset: \n')
print(weather_descriptions)

In [ ]:
weather_desc_counts = data['Weather_Condition'].value_counts(dropna=False,normalize=True)
weather_desc_counts.sort_values(ascending=False).head(25).plot()

Comments & Action Recommendations

### Night/Day Features

There appears to be multiple indicators for day/night that need to be explored.  According to __[Google](https://www.google.com/search?q=different+types+of+twilight&rlz=1C5OZZY_enUS1127US1127&oq=different+types+of+twilight&gs_lcrp=EgZjaHJvbWUyCQgAEEUYORiABDIICAEQABgWGB4yCAgCEAAYFhgeMggIAxAAGBYYHjIICAQQABgWGB4yDQgFEAAYhgMYgAQYigUyDQgGEAAYhgMYgAQYigUyDQgHEAAYhgMYgAQYigUyDQgIEAAYhgMYgAQYigUyCggJEAAYgAQYogTSAQg0NjU3ajFqN6gCALACAA&sourceid=chrome&ie=UTF-8)__, Nautical, Astronomical and Civil all have precise definitions--based on usage--that do not, to me, inform as to how their distinctions contribute to understanding accident patterns.  And as you can see below, there is no clear pattern in how they differ from one another in classifying day vs night.

Ultimately, I think dropping all of them except for Sunrise_Sunset is for the best.  

### Night/Day Features

There appears to be multiple indicators for day/night that need to be explored.  According to __[Google](https://www.google.com/search?q=different+types+of+twilight&rlz=1C5OZZY_enUS1127US1127&oq=different+types+of+twilight&gs_lcrp=EgZjaHJvbWUyCQgAEEUYORiABDIICAEQABgWGB4yCAgCEAAYFhgeMggIAxAAGBYYHjIICAQQABgWGB4yDQgFEAAYhgMYgAQYigUyDQgGEAAYhgMYgAQYigUyDQgHEAAYhgMYgAQYigUyDQgIEAAYhgMYgAQYigUyCggJEAAYgAQYogTSAQg0NjU3ajFqN6gCALACAA&sourceid=chrome&ie=UTF-8)__, Nautical, Astronomical and Civil all have precise definitions--based on usage--that do not, to me, inform as to how their distinctions contribute to understanding accident patterns.  

 And as you can see, there is no clear pattern in how they differ from one another in classifying day vs night.

Ultimately, I think dropping all of them except for Sunrise_Sunset is for the best.

### Infrastructure
Twelve binary features indicating the existance of a traffic design or a PoI.

In [ ]:
infrastructures = ['Amenity','Bump','Crossing','Give_Way',
                   'Junction','No_Exit','Railway','Roundabout',
                   'Station','Stop','Traffic_Calming','Traffic_Signal',
                   'Turning_Loop'
                   ]

Need to verify what the definitions of each of these are...

In [ ]:
data[infrastructures].sum().sort_values(ascending=False)

Traffic_Signal     1143772
Crossing            873763
Junction            571342
Stop                214371
Station             201901
Amenity              96334
Railway              66979
Give_Way             36582
No_Exit              19545
Traffic_Calming       7598
Bump                  3514
Roundabout             249
Turning_Loop             0
dtype: int64

How often do these overlap?

In [ ]:
data[infrastructures].sum(axis=1).value_counts()

In [ ]:
sns.heatmap(data[infrastructures].corr(),vmin=-1,vmax=1,cmap='coolwarm');

# 3-Data Processing

Start with a brand new dataset

In [ ]:
del data
if _SPARK_:
    spark.stop()
else:
    pass

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data_clean = spark.read.csv('US_Accidents_March23.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data_clean = pd.read_csv('US_Accidents_March23_sampled_500k.csv')
    else:
        data_clean = pd.read_csv('US_Accidents_March23.csv')
    

## Drop Unnecessary Columns
Based on prior exploratory work.

## Missing Data

### Ending Longitude/Latitude

### Weather

### Description

### Address

## Outliers

## Create Dates Data
### <a id='datetimes'> Converting dates </a>

In [ ]:
if _SPARK_:
    pass
else:
    data_clean['Start'] = pd.to_datetime(data_clean['Start_Time'],format='mixed')
    data_clean['End'] = pd.to_datetime(data_clean['End_Time'],format='mixed')

And create a new target feature describing the nature of the accident as a function of time.

In [ ]:
data_clean['Time_of_Impact(hr)'] = (data_clean['End'] - data_clean['Start']).dt.seconds/360

In [ ]:
data_clean['Time_of_Impact(hr)'].describe()

## Clean up Bad Data Identified in Prior Exploration

### Time

### Distance

## Feature Engineering

### Partition the time data

### Weekend

### Holidays
Per __[Google](https://www.google.com/search?client=safari&rls=en&q=holidays+with+most+traffic&ie=UTF-8&oe=UTF-8)__, there are a number of holidays in the US with the highest amount of traffic.  (I subbed NYE in for XMas.)

### Rush Hour Indicator

__[Per Google:  ](https://www.google.com/search?client=safari&rls=en&q=rush+hour+typically&ie=UTF-8&oe=UTF-8)__ The morning rush hour begins around 6a, peaking between 7a & 9a, and eases off by 10a.  While afternoon/evening rush hour begins around 3p, peaking between 4p and 6p, and eases off by 7p.

### Attach Location-Based Data to DF

Resources

https://www.faa.gov/air_traffic/publications/atpubs/cnt_html/appendix_a.html


https://apps.bea.gov/regional/geography.htm

https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States

https://www2.census.gov/geo/pdfs/reference/GARM/Ch6GARM.pdf


https://en.wikipedia.org/wiki/Megaregions_of_the_United_States

### Consider Street Type Feature(s)

## Saveout Data

In [ ]:
if _LITE_SWITCH_:
    data_clean.to_csv('AccidentData_Sampled_Clean.csv')
else:
    data_clean.to_csv('AccidentData_Clean.csv')

In [ ]:
del data_clean
if _SPARK_:
    spark.stop()
else:
    pass

# Full EDA
Now with cleaned up data with additional features, there are still some areas of exploration that I want to cover to help inform A) possible other features and B) the statistical analyses.

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        data = pd.read_csv('AccidentData_Clean.csv')
    

In [ ]:
if _SPARK_:
    data.printSchema()
    print("Features: ",len(data.columns))
    print("Entries:  ",data.count())
else:
    data.info()

In [ ]:
_TARGET_ = ['Severity']
_NUMERICS_ = ['Distance(mi)','Time_of_Impact(hr)','Temperature(F)',
              'Wind_Chill(F)','Humidity(%)','Pressure(in)',
              'Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']

## Nature of Incident Variables

## Relationships between Continous Variables

In [ ]:
if _SPARK_:
    corr_data = data.select(_NUMERICS_+['Severity'])
    col_names = corr_data.columns
    features = corr_data.rdd.map(lambda row: row[0:]) 
    corrs = Statistics.corr(features, method="pearson")
else:
    corrs = data[_NUMERICS_].corr()
    print(corrs)
   

In [ ]:
sns.heatmap(corrs,vmin=-1,vmax=1,cmap='coolwarm');

Comments

## Dates

In [ ]:
if _SPARK_:
    accdidents_year = data.stat.crosstab('Year','Severity')
else:
    accdidents_year = pd.crosstab(data['Year'],data['Severity'])

    accdidents_year['Total']=accdidents_year.sum(axis=1)
    accdidents_year['Average']=accdidents_year.apply(lambda r:sum(i*r[i] for i in range(1,5))/r['Total'],axis=1)
    accidents_calendar = pd.pivot_table(data,columns='Year',index='Month',values='Severity')

In [ ]:
display(accdidents_year)
print('*'*75)
display(accidents_calendar)

Comments & Concerns

Other Cross Tabs

### Infrastructure

In [ ]:
infrastructure_count = pd.DataFrame(columns=infrastructures)

### Weather Conditions

In [ ]:
weather_descriptions = data.Weather_Condition.unique()
print(f'There are {weather_descriptions.shape[0]} ways that the weather is described in this dataset: \n')
print(weather_descriptions)

There are 145 ways that the weather is described in this dataset: 

['Light Rain' 'Overcast' 'Mostly Cloudy' 'Rain' 'Light Snow' 'Haze'
 'Scattered Clouds' 'Partly Cloudy' 'Clear' 'Snow'
 'Light Freezing Drizzle' 'Light Drizzle' 'Fog' 'Shallow Fog' 'Heavy Rain'
 'Light Freezing Rain' 'Cloudy' 'Drizzle' nan 'Light Rain Showers' 'Mist'
 'Smoke' 'Patches of Fog' 'Light Freezing Fog' 'Light Haze'
 'Light Thunderstorms and Rain' 'Thunderstorms and Rain' 'Fair'
 'Volcanic Ash' 'Blowing Sand' 'Blowing Dust / Windy' 'Widespread Dust'
 'Fair / Windy' 'Rain Showers' 'Mostly Cloudy / Windy'
 'Light Rain / Windy' 'Hail' 'Heavy Drizzle' 'Showers in the Vicinity'
 'Thunderstorm' 'Light Rain Shower' 'Light Rain with Thunder'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'T-Storm'
 'Heavy Thunderstorms and Rain' 'Thunder' 'Heavy T-Storm' 'Funnel Cloud'
 'Heavy T-Storm / Windy' 'Blowing Snow' 'Light Thunderstorms and Snow'
 'Heavy Snow' 'Low Drifting Snow' 'Light Ice Pellets' 'Ice Pellets'
 'Squal

In [ ]:
description_counts = data.Weather_Condition.value_counts(dropna=False,normalize=True)
description_counts.sort_values(ascending=False)

To use this feature, we need to consolidate somehow.  But first...

### Exploration of Weather Condition Consolidation

Is there a way, we can use the continous features to create a consolidated but meaningful feature which maybe can improve on the categorical weather feature. 

#### PCA

#### K-Means Clustering

#### Decision

### Final Saveout

In [ ]:
if _LITE_SWITCH_:
    data_clean.to_csv('AccidentData_Sampled_Clean_2.csv')
else:
    data_clean.to_csv('AccidentData_Clean_2.csv')

In [ ]:
del data_clean
if _SPARK_:
    spark.stop()
else:
    pass

# Statistical Analyses

Description of the three primary tests that are going to be run for all of these categorical features...

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean_2.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean_2.csv')
    else:
        data = pd.read_csv('AccidentData_Clean_2.csv')
    

## Nature-of-Accident Features vs. Each Other

Comments

In [ ]:
del model_distVseverity, model_timeVseverity
del anova_table_distVSeverity, anova_table_timeVseverity
del p_val

## Weather-Related


In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean_2.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean_2.csv')
    else:
        data = pd.read_csv('AccidentData_Clean_2.csv',usecols = ['Weather_Condition','Distance(mi)','Time_of_Impact(hr)'])
    

## Calendar-Related

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean_2.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean_2.csv')
    else:
        tgt_cols = ['Month','Quarter','Year','DayofWeek','Weekend','Holiday']
        tgt_cols = tgt_cols + ['Hour','Morning_Rush','Evening_Rush','Sunrise_Sunset']
        data = pd.read_csv('AccidentData_Clean.csv',usecols = tgt_cols+['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

### YoY Difference?

Comments

### Quarterly Difference?

Comments

### Monthly ?

Comments

### Day of Week?

Comments

### Weekends?

Comments

### Holidays?

Comments

#### Holidays vs. Weekends?

Comments

## Time of Day?

### Hourly?

Comments

### Rush Hours?


### Day/Night?

Comments

## Location-Related

In [5]:
if _LITE_SWITCH_:
    pass
elif _SPARK_:
    pass
else:
    try:
        del data
        del reports, stats
    except:
        pass
    

NameError: name '_LITE_SWITCH_' is not defined

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        tgt_cols = ['State']
        data = pd.read_csv('AccidentData_Clean.csv',usecols = tgt_cols + ['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

### By State

### Infrastructure

In [ ]:
infrastructures = ['Amenity','Bump','Crossing','Give_Way',
                   'Junction','No_Exit','Railway','Roundabout',
                   'Station','Stop','Traffic_Calming','Traffic_Signal',
                   'Turning_Loop'
                   ]

In [ ]:
if _SPARK_:
    spark = SparkSession.builder.appName("Accident Data Project").getOrCreate()
    data = spark.read.csv('AccidentData_Clean.csv',header=True,inferSchema=True)
else:
    if _LITE_SWITCH_:
        data = pd.read_csv('AccidentData_Sampled_Clean.csv')
    else:
        data = pd.read_csv('AccidentData_Clean.csv',usecols = infrastructures + ['Severity','Distance(mi)','Time_of_Impact(hr)'])
    

Commentary

# 6-Advanced Analysis

# 7-Insights & Conclusions

## Basic Insights

### Annual Distinctions

Number of Accidents; Distribution of Severities; Length of Impact

### Temporal & Spatial Considerations

How do accident counts relate to different times of the day and for different region types (urban vs rural)?

### Weather Considerations
Does certain weather conditions produce more severe incidents?  

## Severity Prediction

## Advanced (Wish List) Analysis

### Unusual Weather 

Does driving in unexpected weather--based on area, time of year and/or both--create a higher likelihood of an accident.

### Famous Highways

## Highway's Near Urban Areas

For cross-state

### Naturual Language

Examination of the description feature.

### Safety Infrastructure
Does certain road infrastructure projects help reduce the number of incidents?

## New Traffic Pattern

Does the existence of a new traffic pattern in the area increase the likelihood of an accident?

## Recent Accident Indicator

Does the presence of one accident, predict another.